In [1]:
import pandas as pd
import numpy as np
import sway_utils.metrics as sm

In [2]:
FPS = 30.0  # Kinect is 30 Hz


def compute_sway_metrics_from_paths(ml_raw, ap_raw, fs: float = FPS):
    """
    Replicates the logic of calculate_sway_from_recording() for Kinect,
    but for a plain ML/AP time series and WITHOUT plotting.

    ml_raw, ap_raw: np.array of CoM in *cm* (as in the example code).
    """
    ml_raw = np.asarray(ml_raw, dtype=float)
    ap_raw = np.asarray(ap_raw, dtype=float)

    # 1) Filter signal like calculate_RD() does for Kinect
    #    Note: fc=8 in that function.
    ml_filt, ap_filt = sm.filter_signal(ml_raw, ap_raw, fc=8, fs=fs)

    # 2) Mean-centred absolute deviations (same as ML/AP in calculate_RD)
    mean_ml = np.mean(ml_filt)
    mean_ap = np.mean(ap_filt)

    ML = np.abs(ml_filt - mean_ml)
    AP = np.abs(ap_filt - mean_ap)
    RD = np.sqrt(ML**2 + AP**2)

    recording_length = len(RD)
    if recording_length == 0:
        raise ValueError("Empty ML/AP series")

    # 3) MDIST and RDIST (identical formulas)
    MDIST_ML = np.sum(ML) / recording_length
    MDIST_AP = np.sum(AP) / recording_length
    MDIST = np.sum(RD) / recording_length

    RDIST_ML = np.sqrt(np.sum(ML**2) / recording_length)
    RDIST_AP = np.sqrt(np.sum(AP**2) / recording_length)
    RDIST = np.sqrt(np.sum(RD**2) / recording_length)

    # 4) TOTEX and fractal dimension (reusing original function)
    TOTEX_ML, TOTEX_AP, TOTEX, FD = sm.calculate_TOTEX(ML, AP)

    # 5) Mean velocity (Kinect → 30 Hz)
    T = recording_length / fs
    MVELO_ML = TOTEX_ML / T
    MVELO_AP = TOTEX_AP / T
    MVELO = TOTEX / T

    # 6) Mean frequency (same as in calculate_sway_from_recording)
    MFREQ_ML = MVELO_ML / (4 * (np.sqrt(2 * MDIST_ML))) if MDIST_ML > 0 else 0.0
    MFREQ_AP = MVELO_AP / (4 * (np.sqrt(2 * MDIST_AP))) if MDIST_AP > 0 else 0.0
    MFREQ = MVELO / (2 * np.pi * MDIST) if MDIST > 0 else 0.0

    # 7) AREA_CE – use the same confidence_ellipse function, but without adding to axes
    #    The function signature is: confidence_ellipse(x, y, ax, ..., return_ellipse=True/False)
    #    With return_ellipse=False it returns: height, width, area, angle
    height, width, AREA_CE, angle = sm.confidence_ellipse(
        ml_filt,
        ap_filt,
        ax=None,
        n_std=1.96,
        return_ellipse=False,
        edgecolor="red",
    )

    return {
        "RDIST_ML": RDIST_ML,
        "RDIST_AP": RDIST_AP,
        "RDIST": RDIST,
        "MDIST_ML": MDIST_ML,
        "MDIST_AP": MDIST_AP,
        "MDIST": MDIST,
        "TOTEX_ML": TOTEX_ML,
        "TOTEX_AP": TOTEX_AP,
        "TOTEX": TOTEX,
        "MVELO_ML": MVELO_ML,
        "MVELO_AP": MVELO_AP,
        "MVELO": MVELO,
        "MFREQ_ML": MFREQ_ML,
        "MFREQ_AP": MFREQ_AP,
        "MFREQ": MFREQ,
        "AREA_CE": AREA_CE,
        "FRAC_DIM": FD,
    }


In [3]:
from sway_utils.recordings import KinectRecording

def extract_sway_for_kinect_recording(
    base_dir: str,
    part_id: int,
    movement_folder: str,
    movement_label: str,
    start_frame: int = 200,
    end_frame: int = 500,
    com_joint_index: int = 25,
) -> dict | None:
    """
    For a given participant & movement, load the KinectRecording,
    extract CoM from joint #25, and compute sway metrics.

    Returns: dict with metrics + identifiers, or None if something is missing.
    """
    str_part = str(part_id)

    skel_root_path = os.path.join(
        base_dir,
        str_part,
        f"{str_part}_{movement_folder}",
        "skel",
    )

    if not os.path.exists(skel_root_path):
        print(f"[WARN] skeleton path not found: {skel_root_path}")
        return None

    try:
        recording = KinectRecording(
            skel_root_path,
            "",                 # depth path not needed for sway
            movement_label,     # just a label, used by the class
            part_id,
        )
    except Exception as e:
        print(f"[ERROR] failed to load KinectRecording for {skel_root_path}: {e}")
        return None

    xyz = recording.stacked_filtered_XYZ_values  # shape [3, 26, frames] usually
    if xyz is None:
        print(f"[WARN] no filtered XYZ values for {skel_root_path}")
        return None

    n_frames = xyz.shape[2]
    start = max(0, start_frame)
    end = min(n_frames, end_frame)
    if end <= start + 1:
        print(f"[WARN] not enough frames ({n_frames}) for {skel_root_path}")
        return None

    # CoM in cm (as in the demo code)
    ml_raw = xyz[0, com_joint_index, start:end] * 100.0  # ML (x)
    ap_raw = xyz[2, com_joint_index, start:end] * 100.0  # AP (z)

    metrics = compute_sway_metrics_from_paths(ml_raw, ap_raw, fs=FPS)

    # Add identifiers
    metrics["part_id"] = part_id
    metrics["movement"] = movement_label
    metrics["start_frame"] = start
    metrics["end_frame"] = end

    return metrics

In [4]:
import os
BASE = "data/sample_set"

row = extract_sway_for_kinect_recording(
    base_dir=BASE,
    part_id=25,
    movement_folder="Quiet-Standing-Eyes-Open",
    movement_label="Quiet-Standing-Eyes-Open",
)

print(row)


{'RDIST_ML': np.float64(0.16574902168413605), 'RDIST_AP': np.float64(0.1754042084355947), 'RDIST': np.float64(0.24132835416951268), 'MDIST_ML': np.float64(0.1295965032064326), 'MDIST_AP': np.float64(0.15620486277413861), 'MDIST': np.float64(0.2208600322118884), 'TOTEX_ML': np.float64(2.3367624833042746), 'TOTEX_AP': np.float64(2.3820361433601724), 'TOTEX': np.float64(3.7371118445647262), 'MVELO_ML': np.float64(0.23367624833042747), 'MVELO_AP': np.float64(0.23820361433601725), 'MVELO': np.float64(0.3737111844564726), 'MFREQ_ML': np.float64(0.11474742289413078), 'MFREQ_AP': np.float64(0.10654328495845933), 'MFREQ': np.float64(0.2693017007165552), 'AREA_CE': np.float64(0.4618488631840259), 'FRAC_DIM': np.float64(3.81147461177036), 'part_id': 25, 'movement': 'Quiet-Standing-Eyes-Open', 'start_frame': 200, 'end_frame': 500}


In [8]:
# --- Config ---
BASE = "/home/timo/mdf/data/kinecal"      # or full kinecal root later
REGISTER_CSV = "data/register_processed.csv"      # adjust path as needed

MOVEMENTS = [
    {
        "folder": "Quiet-Standing-Eyes-Open",
        "label": "Quiet-Standing-Eyes-Open",
        "start_frame": 200,
        "end_frame": 500,
    },
    {
        "folder": "Quiet-Standing-Eyes-Closed",
        "label": "Quiet-Standing-Eyes-Closed",
        "start_frame": 200,
        "end_frame": 500,
    },
    # you can add more movements here:
    # {
    #     "folder": "Quiet-Standing-Eyes-Closed",
    #     "label": "Quiet-Standing-Eyes-Closed",
    #     "start_frame": 200,
    #     "end_frame": 500,
    # },
]

# --- Load participant registry ---
reg = pd.read_csv(REGISTER_CSV)
reg = reg.loc[:, ~reg.columns.str.startswith("Unnamed")]  # drop stray cols

# binary faller label from your group distribution
reg["is_faller"] = reg["group"].isin(["FHs", "FHm"]).astype(int)

rows = []

for _, r in reg.iterrows():
    part_id = int(r["part_id"])

    for mv in MOVEMENTS:
        metrics = extract_sway_for_kinect_recording(
            base_dir=BASE,
            part_id=part_id,
            movement_folder=mv["folder"],
            movement_label=mv["label"],
            start_frame=mv["start_frame"],
            end_frame=mv["end_frame"],
        )
        if metrics is None:
            continue

        # attach labels & demographics
        metrics["group"] = r["group"]
        metrics["age"] = r["age"]
        metrics["sex"] = r["sex"]
        metrics["height"] = r["height"]
        metrics["weight"] = r["weight"]
        metrics["BMI"] = r["BMI"]
        metrics["recorded_in_the_lab"] = r["recorded_in_the_lab"]
        metrics["clinically_at_risk"] = r["clinically-at-risk"]
        metrics["is_faller"] = r["is_faller"]

        rows.append(metrics)

features_df = pd.DataFrame(rows)
print(features_df.head())

# Save for later use in ML
features_df.to_csv("kinecal_sway_features_custom_window.csv", index=False)


[WARN] skeleton path not found: /home/timo/mdf/data/kinecal/7/7_Quiet-Standing-Eyes-Open/skel
[WARN] skeleton path not found: /home/timo/mdf/data/kinecal/11/11_Quiet-Standing-Eyes-Open/skel
[WARN] skeleton path not found: /home/timo/mdf/data/kinecal/11/11_Quiet-Standing-Eyes-Closed/skel
[WARN] skeleton path not found: /home/timo/mdf/data/kinecal/21/21_Quiet-Standing-Eyes-Closed/skel
[WARN] skeleton path not found: /home/timo/mdf/data/kinecal/23/23_Quiet-Standing-Eyes-Open/skel
[WARN] skeleton path not found: /home/timo/mdf/data/kinecal/201/201_Quiet-Standing-Eyes-Closed/skel
[WARN] skeleton path not found: /home/timo/mdf/data/kinecal/307/307_Quiet-Standing-Eyes-Open/skel
[WARN] skeleton path not found: /home/timo/mdf/data/kinecal/312/312_Quiet-Standing-Eyes-Open/skel
   RDIST_ML  RDIST_AP     RDIST  MDIST_ML  MDIST_AP     MDIST  TOTEX_ML  \
0  0.033524  0.272334  0.274389  0.028560  0.236129  0.239285  0.990356   
1  0.129893  0.322521  0.347696  0.122723  0.273363  0.316428  0.878940 